<a href="https://colab.research.google.com/github/BeatriceVaienti/dhCityModeller/blob/master/functionality_test_3D_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **USING DHCITYMODELLER: AUTOMATIC GENERATION OF A 3D CITYJSON**

# **0. REPOSITORY CLONING AND IMPORTS**
Before starting, use this set of cells to clone and install the repository in the Google Colab, and to import the necessary modules.

## 0.0-A Repository cloning while it's private:


In [ ]:
# ONLY UNTIL THE REPO IS PRIVATE!
!wget -q https://raw.githubusercontent.com/tsunrise/colab-github/main/colab_github.py
import colab_github
colab_github.github_auth(persistent_key=True)
!git clone git@github.com:BeatriceVaienti/dhCityModeller.git
# We move inside the folder that we just created and install the repo:
!pip install -q setuptools
%cd /content/dhCityModeller
! pip install .
#geopandas can have issues when installed with pip like we did, so we make sure it is correctly installed:
! pip install --force-reinstall geopandas

Mounted at /content/drive/
Looks that a private key is already created. If you have already push it to github, no action required.
 Otherwise, Please go to https://github.com/settings/ssh/new to upload the following key: 
ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIHwvmnQNtn+uqNJq0CgCpcLeqEqSXUI5eBxjg61eZSOz root@3d3c5dad2a5a

Please use SSH method to clone repo.
Cloning into 'dhCityModeller'...
remote: Enumerating objects: 118, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 118 (delta 55), reused 73 (delta 25), pack-reused 0
Receiving objects: 100% (118/118), 431.15 KiB | 5.07 MiB/s, done.
Resolving deltas: 100% (55/55), done.


## 0.0-B Repository cloning once it's public:

In [ ]:
# only when the repo is public
! git clone https://github.com/BeatriceVaienti/dhCityModeller.git
! pip install git+https://github.com/BeatriceVaienti/dhCityModeller.git@master

## 0.1 Necessary Imports

In [ ]:
import geopandas as gpd
import json
import sys
sys.path.append('../')
import copy
import tqdm
import modules.encoder as encoder
import modules.terrain as terrain

# **1. THE PIPELINE**
Starting from a GeoJSON whose attributes are filled accordingly to the Historical CityJSON extension, we will follow the pipeline that leads to the automatic generation of a 3D CityJSON file containing LOD0, LOD1, and LOD2 geometries.


Every section of the pipeline can be skipped if you already have a file to import and you just want to start from a CityJSON file.


## **1.1 GENERATION OF A 2D CITYJSON**
In the folder /tests/import/ we provide a test dataset that is filled accordingly to our extension. Of course, it is also possible to test our library with your own dataset: just put your GeoJSON in the /tests/import folder after making it compliant with the Historical CityJSON extension. It is necessary to respect the extension in order to have a correct mapping between the fields contained in the original file and the attributes that are then going to be used as parameters for the process of procedural modelling. If you want to be guided in the process of mapping the fields of your files in a compliant way and to use our machine learning tools to fill gaps in your dataset, use our other Colab ...

First, we read the GeoJSON file:

In [ ]:
gdf_test = gpd.read_file("./tests/import/FILLED_DATASET_SUBSET.geojson")

Then, we convert the geojson into a 2D CityJSON file without any modification (the process is a simple mapping between the GeoJSON geometries and tabular-form attributes into a CityJSON geometry and hierarchical-form structure)

In [ ]:
cityjson_2d=encoder.gdf_to_CityJSON(gdf_test) # clean and convert the dataframe to a cityjson file (2D!)
#save the 2D cityjson file
with open('./tests/export/2d_cityjson.json', 'w') as outfile:
    json.dump(cityjson_2d, outfile)

## **1.2 TERRAIN GENERATION**
In this phase we generate the terrain and translate vertically the footprints to the terrain level, obtaining a CityJSON file that contains the terrain and the footprints vertically displaced.

The terrain data can be obtained either using [OpenTopoData](https://www.opentopodata.org) or by using the [Google Elevation API](https://developers.google.com/maps/documentation/elevation/overview). If you decide to use the latter, it is necessary to provide your apikey in the "insert_terrain_in_cityjson" function.

##### If you already have a 2D CityJSON file and you want to import that from file:

In [ ]:
with open('./export/2d_cityjson.json') as json_file:
    cityjson_2d = json.load(json_file)

##### We query the elevation, generate a TIN mesh, move vertically the footprints and embed them in the CityJSON file:

In [ ]:
cityjson2d_terrain = terrain.insert_terrain_in_cityjson(cityjson_2d, service = 'opentopodata', apikey = None , grid_min_distance = 60, tin_max_error = 1)

##### We can now save the file that we obtained:

In [ ]:
#save the file with terrain
with open('./export/2d_cityjson_with_terrain.json', 'w') as outfile:
    json.dump(cityjson2d_terrain, outfile)

## **1.3 PROCEDURAL MODELLING: GENERATION OF THE 3D GEOMETRIES**
We iterate over each object of the cityjson_2d and we use its attributes to create the 3D cityjson. (LOD1 and LOD2 geometry)

##### If you already have a 2D CityJSON file and you want to import that from file:

In [ ]:
with open('./export/2d_cityjson_with_terrain.json') as json_file:
    cityjson2d_terrain = json.load(json_file)

##### We iterate over each object, we make sure that they are of type "Building" and we procedurally create their 3D geometry encoding it in the CityJSON file:

In [ ]:
cityjson_3d= copy.deepcopy(cityjson2d_terrain)
for id in tqdm.tqdm(cityjson_3d['CityObjects'].keys()): #loop over all the buildings
    if cityjson_3d['CityObjects'][id]['type'] == 'Building':
      cityjson_3d = encoder.update_cityjson_geometry(cityjson_3d, id) #update the cityjson file with the 3D geometry of the building


 27%|██▋       | 204/767 [09:13<12:32,  1.34s/it]

In [ ]:
# and save the 3D cityjson file:
with open('./export/3d_cityjson_hip_test.json', 'w') as json_3d:
    json.dump(cityjson_3d, json_3d)